In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from bs4 import BeautifulSoup # library for pulling data out of HTML and XML files
import requests 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_areas_of_London'

In [ ]:
source = requests.get(url)
#soup=BeautifulSoup(source,'lxml')

In [ ]:
london_data = pd.read_html(source.text)


In [ ]:
london_data = london_data[1]
london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [ ]:
type(london_data)

pandas.core.frame.DataFrame

Rename the Columns

In [ ]:
london_data.columns=['Location','Borough','Town','Postalcode','Dial','OS']

Remove the [] and numbers from Boroughs

In [ ]:
# Remove Borough reference numbers with []
london_data['Borough'] = london_data['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

Split the strings to drop an extra entry

In [ ]:
df0 = london_data.drop('Postalcode', axis=1).join(london_data['Postalcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postalcode'))

Only retain Town entries for London

In [ ]:
df1 = df0[df0['Town'].str.contains('LONDON')]

Drop a column

In [ ]:
del df1['Dial']

In [ ]:
df1

,Location,Borough,Town,OS,Postalcode
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W3
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W4
6,Aldgate,City,LONDON,TQ334813,EC3
7,Aldwych,Westminster,LONDON,TQ307810,WC2
...,...,...,...,...,...
521,Woodford,Redbridge,LONDON,TQ405915,E18
522,Woodford Green,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",TQ405915,IG8
525,Woodside Park,Barnet,LONDON,TQ256925,N12
526,Woolwich,Greenwich,LONDON,TQ435795,SE18


In [ ]:
#!pip install geopandas
#!pip install geopy

In [ ]:
locator = Nominatim(user_agent='myGeocoder')


In [ ]:
location = locator.geocode("Marble Arch, london, UK")

In [ ]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 51.51317575, Longitude = -0.15893653525883872


In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['Postalcode'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df1.index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [ ]:
df1

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,altitude
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"(Rumbia, Rivervale Drive, Seng Kang, Singapore...","(1.3914749, 103.9059729, 0.0)",1.391475,103.905973,0.0
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W3,"(W3, 岡崎市, 愛知県, 日本, (34.968282, 137.31433574237...","(34.968282, 137.3143357423729, 0.0)",34.968282,137.314336,0.0
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W4,"(W4, 岡崎市, 愛知県, 日本, (34.96589455, 137.31516305))","(34.96589455, 137.31516305, 0.0)",34.965895,137.315163,0.0
6,Aldgate,City,LONDON,TQ334813,EC3,"(City of London, Greater London, England, EC3,...","(51.51135485415663, -0.08204644757648563, 0.0)",51.511355,-0.082046,0.0
7,Aldwych,Westminster,LONDON,TQ307810,WC2,"(WC2, Park County, Wyoming, United States, (44...","(44.9698976, -111.0129626, 0.0)",44.969898,-111.012963,0.0
...,...,...,...,...,...,...,...,...,...,...
521,Woodford,Redbridge,LONDON,TQ405915,E18,"(上信越自動車道, 藤岡市, 群馬県, 375-0004, 日本, (36.2724718,...","(36.2724718, 139.077708, 0.0)",36.272472,139.077708,0.0
522,Woodford Green,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",TQ405915,IG8,"(London Borough of Waltham Forest, London, Gre...","(51.603696464013645, 0.004747690693558546, 0.0)",51.603696,0.004748,0.0
525,Woodside Park,Barnet,LONDON,TQ256925,N12,"(ドーム前千代崎, 大阪府道173号大阪八尾線, 千代崎三丁目, 西区, 大阪市, 大阪府,...","(34.6714542, 135.4794918, 0.0)",34.671454,135.479492,0.0
526,Woolwich,Greenwich,LONDON,TQ435795,SE18,"(Royal Borough of Greenwich, London, Greater L...","(51.4908330082255, 0.056173599698051385, 0.0)",51.490833,0.056174,0.0


In [ ]:
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['Borough'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df1.index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
df1

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,altitude
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"(Bexley Road, Avery Hill, Royal Borough of Gre...","(51.4510531, 0.0790997, 0.0)",51.451053,0.079100,0.0
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W3,"(Ealing, Hammersmith & West London College, St...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971,0.0
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W4,"(Ealing, Hammersmith & West London College, St...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971,0.0
6,Aldgate,City,LONDON,TQ334813,EC3,"(City of London, Greater London, England, Unit...","(51.5156177, -0.0919983, 0.0)",51.515618,-0.091998,0.0
7,Aldwych,Westminster,LONDON,TQ307810,WC2,"(Westminster, Millbank, City of Westminster, L...","(51.5004439, -0.1265398, 0.0)",51.500444,-0.126540,0.0
...,...,...,...,...,...,...,...,...,...,...
521,Woodford,Redbridge,LONDON,TQ405915,E18,"(Redbridge, Eastern Avenue, Redbridge, London ...","(51.5763203, 0.0454097, 0.0)",51.576320,0.045410,0.0
522,Woodford Green,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",TQ405915,IG8,"(Waltham Road, Woodford, London Borough of Red...","(51.6081792, 0.0526336, 0.0)",51.608179,0.052634,0.0
525,Woodside Park,Barnet,LONDON,TQ256925,N12,"(Chipping Barnet, London, Greater London, Engl...","(51.65309, -0.2002261, 0.0)",51.653090,-0.200226,0.0
526,Woolwich,Greenwich,LONDON,TQ435795,SE18,"(Greenwich, London, Greater London, England, S...","(51.4820845, -0.0045417, 0.0)",51.482084,-0.004542,0.0


In [ ]:
import folium

In [ ]:
map1 = folium.Map(
    location=[51.509865,-0.118092], ##coordinates for London
    tiles='cartodbpositron',
    zoom_start=12,
)


In [ ]:
dfx.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [ ]:
dfx=df1.dropna()

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
path='/content/drive/MyDrive/Python_text/Py_Webscraping/lectures/section5'

In [ ]:
df1.to_csv('/content/drive/MyDrive/Python_text/Py_Webscraping/lectures/section5/londongeo.csv')

In [ ]:
##https://medium.com/@oludayo.oguntoyinbo/the-battle-of-neighbourhood-my-londons-perspective-d363163771e0

In [ ]:
##https://medium.com/analytics-vidhya/a-tale-of-two-cities-clustering-neighborhoods-of-london-and-paris-5328f69cd8b6

In [ ]:
##https://simplemaps.com/data/sg-cities

In [ ]:
## London infomration: https://data.london.gov.uk/dataset/london-borough-profiles

In [ ]:
##https://towardsdatascience.com/geocode-with-python-161ec1e62b89